In [29]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
im = Image.open('chad/1985.jpg')
im.show()

In [44]:
import os
import numpy as np

In [45]:
import os
sat_images = {}

for filename in os.listdir('chad/'):
    if filename.endswith(".jpg"):
        sat_images[filename] = Image.open('chad/' + filename)

In [46]:
sat_images

{'1999.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E240>,
 '1989.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E2B0>,
 '1988.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E320>,
 '2003.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E390>,
 '2002.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E400>,
 '2016.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E198>,
 '2000.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E470>,
 '2014.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E4E0>,
 '2015.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E550>,
 '2001.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4800x3251 at 0x112B0E5C0>,
 '2005.jpg': <PIL.JpegImagePlugin.JpegImageFile im

In [48]:
for year in sat_images:
    im_array = np.array(sat_images[year])
    for row in im_array:
        for pixel in row:
            if pixel[2] < 125 and pixel[2] > 105 and pixel[0] > 115 and pixel[0] < 150 and pixel[1] > 115 and pixel[1] < 150:
                pixel[0] = 0
                pixel[1] = 0
                pixel[2] = 245
                
    im = Image.fromarray(im_array)
    im.save(year + "processed.jpg")

TypeError: iteration over a 0-d array